In [67]:
#!/usr/bin/python
# coding: utf8

'''
Created on Mar 24, 2011
Update  on 2017-05-18
Ch 11 code
Author: Peter/片刻
GitHub: https://github.com/apachecn/AiLearning'''
print(__doc__)
from numpy import *

# 加载数据集
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

# 创建集合 C1。即对 dataSet 进行去重，排序，放入 list 中，然后转换所有的元素为 frozenset
def createC1(dataSet):
    """createC1（创建集合 C1）
    Args:
        dataSet 原始数据集
    Returns:
        frozenset 返回一个 frozenset 格式的 list
    """

    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                # 遍历所有的元素，如果不在 C1 出现过，那么就 append
                C1.append([item])
    # 对数组进行 `从小到大` 的排序
    # print 'sort 前=', C1
    C1.sort()
    # frozenset 表示冻结的 set 集合，元素无改变；可以把它当字典的 key 来使用
    # print 'sort 后=', C1
    # print 'frozenset=', map(frozenset, C1)
    return C1

# 计算候选数据集 CK 在数据集 D 中的支持度，并返回支持度大于最小支持度（minSupport）的数据
''' 计算候选数据集CK在数据集D中的支持度，返回大于最小支持度的数据'''
def scanD(D,Ck,minSupport):
    # ssCnt 临时存放所有候选项集和频率.
    ssCnt = {}
    for tid in D:
        #print('1:',tid)
        for can in map(frozenset,Ck):      #每个候选项集can
            #print('2:',can.issubset(tid),can,tid)
            if can.issubset(tid):
                if not can in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] +=1

    numItems = float(len(D)) # 所有项集数目
    # 满足最小支持度的频繁项集
    retList  = []
    # 满足最小支持度的频繁项集和频率
    supportData = {}

    for key in ssCnt:
        support = ssCnt[key]/numItems   #除以总的记录条数，即为其支持度
        if support >= minSupport:
            retList.insert(0,key)       #超过最小支持度的项集，将其记录下来。
        supportData[key] = support
    return retList, supportData


Created on Mar 24, 2011
Update  on 2017-05-18
Ch 11 code
Author: Peter/片刻
GitHub: https://github.com/apachecn/AiLearning


In [62]:
if __name__ == "__main__":
    data = loadDataSet()
    Cset = createC1(data)
    popularSet, setSupportData = scanD(data,Cset,0.5)

1: [1, 3, 4]
2: True frozenset({1}) [1, 3, 4]
2: False frozenset({2}) [1, 3, 4]
2: True frozenset({3}) [1, 3, 4]
2: True frozenset({4}) [1, 3, 4]
2: False frozenset({5}) [1, 3, 4]
1: [2, 3, 5]
2: False frozenset({1}) [2, 3, 5]
2: True frozenset({2}) [2, 3, 5]
2: True frozenset({3}) [2, 3, 5]
2: False frozenset({4}) [2, 3, 5]
2: True frozenset({5}) [2, 3, 5]
1: [1, 2, 3, 5]
2: True frozenset({1}) [1, 2, 3, 5]
2: True frozenset({2}) [1, 2, 3, 5]
2: True frozenset({3}) [1, 2, 3, 5]
2: False frozenset({4}) [1, 2, 3, 5]
2: True frozenset({5}) [1, 2, 3, 5]
1: [2, 5]
2: False frozenset({1}) [2, 5]
2: True frozenset({2}) [2, 5]
2: False frozenset({3}) [2, 5]
2: False frozenset({4}) [2, 5]
2: True frozenset({5}) [2, 5]


In [63]:
popularSet

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [64]:
''' Apriori算法：输入频繁项集列表Lk，输出所有可能的候选项集 Ck'''
def aprioriGen(Lk, k):
    retList = [] # 满足条件的频繁项集
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[: k-2]
            L2 = list(Lk[j])[: k-2]
            # print '-----i=', i, k-2, Lk, Lk[i], list(Lk[i])[: k-2]
            # print '-----j=', j, k-2, Lk, Lk[j], list(Lk[j])[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

In [65]:
aprioriGen(popularSet, 2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

In [66]:
'''找出数据集中支持度不小于最小支持度的候选项集以及它们的支持度即频繁项集。
算法思想：首先构建集合C1，然后扫描数据集来判断这些只有一个元素的项集是否满足最小支持度。满足最小支持度要求的项集构成集合L1。然后L1 中的元素相互组合成C2，C2再进一步过滤变成L2，以此类推，直到C_n的长度为0时结束，即可找出所有频繁项集的支持度。
返回：L 频繁项集的全集
      supportData 所有元素和支持度的全集
'''
def apriori(dataSet, minSupport=0.5):
    # C1即对dataSet去重排序，然后转换所有的元素为frozenset
    C1 = createC1(dataSet)
    # 对每一行进行 set 转换，然后存放到集合中
    D = list(map(set, dataSet))
    # 计算候选数据集C1在数据集D中的支持度，并返回支持度大于minSupport 的数据
    L1, supportData = scanD(D, C1, minSupport)
    # L 加了一层 list, L一共 2 层 list
    L = [L1];k = 2
    # 判断L第k-2项的数据长度是否>0即频繁项集第一项。第一次执行时 L 为 [[frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])]]。L[k-2]=L[0]=[frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])]，最后面 k += 1
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k) # 例如: 以 {0},{1},{2} 为输入且 k = 2 则输出 {0,1}, {0,2}, {1,2}. 以 {0,1},{0,2},{1,2} 为输入且 k = 3 则输出 {0,1,2}

        # 返回候选数据集CK在数据集D中的支持度大于最小支持度的数据
        Lk, supK = scanD(D, Ck, minSupport)
        # 保存所有候选项集的支持度，如果字典没有就追加元素，如果有就更新元素
        supportData.update(supK)
        if len(Lk) == 0:
            break
        # Lk 表示满足频繁子项的集合，L 元素在增加，例如:
        # l=[[set(1), set(2), set(3)]]
        # l=[[set(1), set(2), set(3)], [set(1, 2), set(2, 3)]]
        L.append(Lk)
        k += 1
    return L, supportData

In [68]:
'''测试频繁项集生产'''
def testApriori():
    # 加载测试数据集
    dataSet = loadDataSet()
    print ('dataSet: ', dataSet)

    # Apriori 算法生成频繁项集以及它们的支持度
    L1, supportData1 = apriori(dataSet, minSupport=0.7)
    print ('L(0.7): ', L1)
    print ('supportData(0.7): ', supportData1)

    print ('->->->->->->->->->->->->->->->->->->->->->->->->->->->->')

    # Apriori 算法生成频繁项集以及它们的支持度
    L2, supportData2 = apriori(dataSet, minSupport=0.5)
    print ('L(0.5): ', L2)
    print ('supportData(0.5): ', supportData2)

In [69]:
testApriori()

dataSet:  [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
L(0.7):  [[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})]]
supportData(0.7):  {frozenset({1}): 0.5, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({2}): 0.75, frozenset({5}): 0.75, frozenset({2, 5}): 0.75, frozenset({3, 5}): 0.5, frozenset({2, 3}): 0.5}
->->->->->->->->->->->->->->->->->->->->->->->->->->->->
L(0.5):  [[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})], [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})], [frozenset({2, 3, 5})]]
supportData(0.5):  {frozenset({1}): 0.5, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({2}): 0.75, frozenset({5}): 0.75, frozenset({1, 3}): 0.5, frozenset({2, 5}): 0.75, frozenset({3, 5}): 0.5, frozenset({2, 3}): 0.5, frozenset({1, 5}): 0.25, frozenset({1, 2}): 0.25, frozenset({2, 3, 5}): 0.5}
